<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173_Fall2025/blob/main/F25_Class_04_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

##### **Module 4: ChatGPT and Large Language Models**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Biology, Health and the Environment](https://sciences.utsa.edu/bhe/), [UTSA](https://www.utsa.edu/)

### Module 4 Material

* Part 4.1: Introduction to Transformers and Accessing ChatGTP
* **Part 4.2: LLM Memory. Embedding and Prompt Engineering**
* Part 4.3: Generative AI
* Part 4.4: Text to Images with Stable Diffusion


## Google CoLab Instructions

You MUST run the following code cell to get credit for this class lesson. By running this code cell, you will map your GDrive to /content/drive and print out your Google GMAIL address. Your Instructor will use your GMAIL address to verify the author of this class lesson.

In [1]:
# You must run this cell first
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    COLAB = True
    print("Note: Using Google CoLab")
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    COLAB = False

Mounted at /content/drive
Note: Using Google CoLab
david.senseman@gmail.com


Make sure your GMAIL address is included as the last line in the output above.

### Install Custom Functions

Run the cell below to load custom functions used in this lesson.

In [ ]:
# Simple function to print out elasped time
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

# **LLM Memory**

Human minds have both long-term and short-term memory. Long-term memory is what the human has learned throughout their lifetime. Short-term memory is what a human has only recently discovered in the last minute or so. For humans, learning is converting short-term memory into long-term memory that we will retain.

This process works somewhat differently for a LLM. Long-term memory was the weight of the neural network when it was initially trained or finetuned. Short-term memory is additional information that we wish the LLM to retain from previous prompts. For example, if the first prompt is "My name is Jeff", the LLM will likely tell you hello and repeat your name. However, the LLM will not know the answer if the second prompt is "What is my name." without adding a memory component.

These memory objects, which LangChain provides, provide a sort of short-term memory. It is important to note that these objects are not affecting the long-term memory of the LLM, and once you discard the memory object, the LLM will forget. Additionally, the memory object can only hold so much information; newer information may replace older information once it is filled.

One important point to remember is that LLM's only have their input prompt. To provide such memory, these objects are appending anything we wish the LLM to remember to the input prompt. This section will see two ways to augment the prompt with previous information: a buffer and a summary. The buffer prepends a script of the last conversation up to this point. The summary approach keeps a consistently updated summary paragraph of the conversation.

### Install LongChain

In [2]:
!pip install langchain langchain_openai > /dev/null

## **Obtaining an OpenAI API Key**

In order to delve into the practical exercises and code demonstrations within this section, students will need to obtain an **OpenAI API key**. This key grants access to OpenAI's services, including the ChatGPT functionality we'll be exploring. It's important to note that there is a nominal cost associated with the usage of this key, depending on the volume and intensity of requests made to OpenAI's servers.

To obtain an OpenAI API key, access this [site](https://platform.openai.com/apps).

In [3]:
# This is the model you will generally use for this class
LLM_MODEL = 'gpt-3.5-turbo-1106'

We begin with a very basic query to LangChain, we ask LangChain what are the 5 largest cities in the USA.


In [6]:
from google.colab import userdata
from langchain_openai import OpenAI, ChatOpenAI

# Retrieve the OpenAI API key and store it in a variable
OPENAI_KEY = userdata.get('OPENAI_KEY')

# Ensure that the API key is correctly set
if not OPENAI_KEY:
    raise ValueError("OpenAI API key is not set. Please check if you have stored the API key in userdata.")

LLM_MODEL = 'gpt-3.5-turbo-1106'

# Initialize the OpenAI LLM (Language Learning Model) with your API key using ChatOpenAI
llm = ChatOpenAI(openai_api_key=OPENAI_KEY, model=LLM_MODEL, temperature=0.7)

# Define the question
question = "What is the largest university in San Antonio, Texas?"

# Use Langchain to call the OpenAI chat API
response = llm.invoke(question)

# Print the response
print(response.content)


The largest university in San Antonio, Texas is the University of Texas at San Antonio (UTSA), with an enrollment of over 32,000 students.


## Conversation Buffer Window Memory

The LangChain library includes a conversation object named **ConversationChain**; this object facilitates an ongoing conversation with an LLM. For any conversation object, you must also specify a memory. For this first example, we will use the **ConversationBufferWindowMemory** object. This object keeps a transcript of the most recent conversation to reference. This memory allows the conversation object to remember what you have asked or told it and its responses to you.

In [7]:
#
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

<ipython-input-7-3853fa6c43bb>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory()
<ipython-input-7-3853fa6c43bb>:6: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


We can now have a conversation with the LLM.



In [8]:
#
conversation.predict(input="Hi, my name is David")

"Hello, David! It's nice to meet you. My name is AI-3000. I have access to a vast amount of information and can help answer any questions you may have. How can I assist you today?"

This newly constructed prompt can now perform the intended task of translation.

In [9]:
conversation.predict(input="What is my name?")

'Your name is David.'

We can have a look at what the memory now contains.

In [10]:
conversation.memory.load_memory_variables({})

{'history': "Human: Hi, my name is David\nAI: Hello, David! It's nice to meet you. My name is AI-3000. I have access to a vast amount of information and can help answer any questions you may have. How can I assist you today?\nHuman: What is my name?\nAI: Your name is David."}

## **Custom Conversation Bots**

You can define the prompt template for a conversationbot. This technique allows you to create a bot with a name and perform a specialized task. In this case, we created a bot named "WashU Assistant" that we designed to help students.

In [12]:
# Original code
# Now we can override it and set it to "AI Assistant"
from langchain.prompts.prompt import PromptTemplate

template = """The following is a friendly conversation between a human and an
AI to assist UTSA Students. The AI should stick to topics
about The University of Texas at San Antonion (UTSA). If the AI does not know the answer to a question,
it should suggest the student speak to their advisor.

Current conversation:
{history}
Human: {input}
UTSA Assistant:"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=False,
    memory=ConversationBufferWindowMemory(ai_prefix="UTSA Assistant"),
)

We can now have a conversation with the UTSA assistant bot.



In [13]:
# Orignal code
conversation.predict(input="Where is the bookstore?")

"The bookstore is located at the Main Campus in the University Center. It's a great place to get textbooks, UTSA gear, and school supplies. If you need more specific directions, I suggest checking the campus map or speaking with someone at the information desk."

Another question.


In [14]:
conversation.predict(input="What is a nice quiet area to study?")


'The John Peace Library is a great place to study. It has quiet study areas, group study rooms, and access to a variety of resources. There are also other libraries on campus, such as the Downtown Campus Library and the Applied Engineering and Technology Library, that offer quiet study spaces. If you need help finding the perfect spot, I suggest speaking with a librarian for recommendations.'

Often you will have multiple components in langchain that you must call in a "chain", to do this you can construct a chain.


In [15]:
conversation.predict(input="Which of these is closest to the bookstore?")


'The closest library to the bookstore is the John Peace Library, as it is also located on the Main Campus in close proximity to the University Center where the bookstore is located. If you need assistance with finding the specific location, I recommend speaking with someone at the bookstore or the information desk for more accurate directions.'

In [16]:
conversation.predict(input="What is the meaning of life.")

'I am here to assist UTSA students with questions related to the university. If you have philosophical or existential questions, I recommend seeking guidance from a trusted mentor, counselor, or advisor.'

We can have a look at what the memory now contains.

In [17]:
conversation.memory.load_memory_variables({})

{'history': "Human: Where is the bookstore?\nUTSA Assistant: The bookstore is located at the Main Campus in the University Center. It's a great place to get textbooks, UTSA gear, and school supplies. If you need more specific directions, I suggest checking the campus map or speaking with someone at the information desk.\nHuman: What is a nice quiet area to study?\nUTSA Assistant: The John Peace Library is a great place to study. It has quiet study areas, group study rooms, and access to a variety of resources. There are also other libraries on campus, such as the Downtown Campus Library and the Applied Engineering and Technology Library, that offer quiet study spaces. If you need help finding the perfect spot, I suggest speaking with a librarian for recommendations.\nHuman: Which of these is closest to the bookstore?\nUTSA Assistant: The closest library to the bookstore is the John Peace Library, as it is also located on the Main Campus in close proximity to the University Center where

## **Conversation Summary Memory**

Now, let's look at using a slightly more complex type of memory, the ConversationSummaryMemory object. This type of memory creates a summary of the conversation over time. This memory can be helpful for condensing information from the conversation over time. Conversation summary memory summarizes the conversation and stores the current summary in memory. You can use this memory to inject the conversation summary so far into a prompt/chain. This memory is most useful for more extended conversations, where keeping the past message history in the prompt verbatim would take up too many tokens.

In [18]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

<ipython-input-18-c71968a3d896>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [20]:
conversation.predict(input="I am a computational biologist, what do you do for a living?")

'I am an artificial intelligence designed to assist and provide information to users. My main job is to constantly learn and improve my capabilities through interactions with users like you. I am programmed to analyze data, answer questions, and perform tasks to the best of my ability.'

In [21]:
conversation.memory.load_memory_variables({})

{'history': 'The human asks the AI what it does for a living. The AI explains that it is an artificial intelligence designed to assist and provide information to users, constantly learning and improving its capabilities through interactions. It also mentions that it is programmed to analyze data, answer questions, and perform tasks to the best of its ability.'}

## **Lesson Turn-in**

When you have completed and run all of the code cells, use the **File --> Print.. --> Save to PDF** to generate a PDF of your Colab notebook. Save your PDF as `Copy of Class_04_2.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.

## **Lizard Tail**

## **Attention Is All You Need**

![__](https://upload.wikimedia.org/wikipedia/commons/8/8f/The-Transformer-model-architecture.png)

**"Attention Is All You Need"** is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer, based on the attention mechanism proposed in 2014 by Bahdanau et al. It is considered a foundational paper in modern artificial intelligence, as the transformer approach has become the main architecture of large language models like those based on GPT. At the time, the focus of the research was on improving Seq2seq techniques for machine translation, but the authors go further in the paper, foreseeing the technique's potential for other tasks like question answering and what is now known as multimodal Generative AI.

The paper's title is a reference to the song "All You Need Is Love" by the Beatles. The name "Transformer" was picked because Jakob Uszkoreit, one of the paper's authors, liked the sound of that word.

An early design document was titled "Transformers: Iterative Self-Attention and Processing for Various Tasks", and included an illustration of six characters from the Transformers animated show. The team was named Team Transformer.

Some early examples that the team tried their Transformer architecture on included English-to-German translation, generating Wikipedia articles on "The Transformer", and parsing. These convinced the team that the Transformer is a general purpose language model, and not just good for translation.

As of 2024, the paper has been cited more than 140,000 times.

**Authors**

The authors of the paper are: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan Gomez, Lukasz Kaiser, and Illia Polosukhin. All eight authors were "equal contributors" to the paper; the listed order was randomized. The Wired article highlights the group's diversity:

Six of the eight authors were born outside the United States; the other two are children of two green-card-carrying Germans who were temporarily in California and a first-generation American whose family had fled persecution, respectively.

**Methods discussed & introduced**

The paper is most well known for the introduction of the Transformer architecture, which forms the underlying architecture for most forms of modern Large Language Models (LLMs). A key reason for why the architecture is preferred by most modern LLMs is the parallelizability of the architecture over its predecessors. This ensures that the operations necessary for training can be accelerated on a GPU allowing both faster training times and models of bigger sizes to be trained.

The following mechanisms were introduced by the paper as part of the development of the transformer architecture.

**Scaled dot-product attention & self-attention**

The use of the scaled dot-product attention and self-attention mechanism instead of an RNN or LSTM (which rely on recurrence instead) allow for better performance as described in the following paragraph.

Since the model relies on Query (Q), Key (K) and Value (V) matrices that come from the same source itself (i.e. the input sequence / context window), this eliminates the need for RNNs completely ensuring parallelizability for the architecture. This differs from the original form of the Attention mechanism introduced in 2014. Additionally, the paper also discusses the use of an additional scaling factor that was found to be most effective with respect to the dimension of the key vectors.

In the specific context of translation which the paper focused on, the Query and Key matrices are usually represented in embeddings corresponding to the source language while the Value matrix corresponds to the target language.

**Multi-head attention**

In the self-attention mechanism, queries (Q), keys (K), and values (V) are dynamically generated for each input sequence (limited typically by the size of the context window), allowing the model to focus on different parts of the input sequence at different steps. Multi-head attention enhances this process by introducing multiple parallel attention heads. Each attention head learns different linear projections of the Q, K, and V matrices. This allows the model to capture different aspects of the relationships between words in the sequence simultaneously, rather than focusing on a single aspect.

By doing this, multi-head attention ensures that the input embeddings are updated from a more varied and diverse set of perspectives. After the attention outputs from all heads are calculated, they are concatenated and passed through a final linear transformation to generate the output.

**Positional encoding**

Since the Transformer model is not a seq2seq model and does not rely on the sequence of the text in order to perform encoding and decoding, the paper relied on the use of sine and cosine wave functions to encode the position of the token into the embedding.

** Historical context**

For many years, sequence modelling and generation was done by using plain recurrent neural networks (RNNs). A well-cited early example was the Elman network (1990). In theory, the information from one token can propagate arbitrarily far down the sequence, but in practice the vanishing-gradient problem leaves the model's state at the end of a long sentence without precise, extractable information about preceding tokens.

A key breakthrough was LSTM (1995), a RNN which used various innovations to overcome the vanishing gradient problem, allowing efficient learning of long-sequence modelling. One key innovation was the use of an attention mechanism which used neurons that multiply the outputs of other neurons, so-called multiplicative units. Neural networks using multiplicative units were later called sigma-pi networks or higher-order networks. LSTM became the standard architecture for long sequence modelling until the 2017 publication of Transformers. However, LSTM still used sequential processing, like most other RNNs. Specifically, RNNs operate one token at a time from first to last; they cannot operate in parallel over all tokens in a sequence.

Modern Transformers overcome this problem, but unlike RNNs, they require computation time that is quadratic in the size of the context window. The linearly scaling fast weight controller (1992) learns to compute a weight matrix for further processing depending on the input. One of its two networks has "fast weights" or "dynamic links" (1981). A slow neural network learns by gradient descent to generate keys and values for computing the weight changes of the fast neural network which computes answers to queries. This was later shown to be equivalent to the unnormalized linear Transformer.